In [1]:
%runfile testcong.py

## Read all optimal curves from the internal database (conductors from $1$ to $400000$) and make a hash table from the $a_p$ for $30$ primes (the first $30$ over $400000$).  This takes about 47 minutes so after doing it once we save the result, and later runs will read this (which takes a few seconds: the saved file is 52M).

In [2]:
try:
    hashtab_13_30 = load('hashtab_13_30')
except IOError:
    hashtab_13_30 = make_hash(13,1,400000,30)
    save(hashtab_13_30, 'hashtab_13_30')

510g1
862a1
1155i1
1464a1
1755f1
2046c1
2337c1
2597c1
2850w1
3138e1
3392p1
3660b1
3906g1
4167a1
4424a1
4680e1
4928bb1
5184d1
5436a1
5680h1
5910a1
6138g1
6381a1
6594r1
6840b1
7067a1
7325c1
7575h1
7812f1
8064p1
8300a1
8549c1
8789d1
9030bd1
9270v1
9510p1
9768e1
9994a1
10224l1
10465a1
10703d1
10913c1
11152l1
11389a1
11622a1
11856x1
12090bk1
12321e1
12558a1
12810v1
13050f1
13256a1
13485i1
13725a1
13950bt1
14175z1
14425b1
14658i1
14892j1
15120k1
15344b1
15584i1
15808b1
16048bc1
16296c1
16533d1
16731c1
16980c1
17220c1
17442k1
17664b1
17880m1
18135e1
18333h1
18564c1
18800bs1
19032o1
19264i1
19491g1
19698d1
19950c1
20175a1
20400cv1
20650o1
20850e1
21075b1
21294ch1
21525i1
21735e1
21987d1
22218bd1
22464i1
22695i1
22906e1
23136h1
23352h1
23600a1
23816h1
24050q1
24273l1
24490f1
24735b1
24961a1
25200u1
25408c1
25608f1
25861e1
26082q1
26325b1
26550cm1
26802a1
27027r1
27225bw1
27475d1
27720e1
27930t1
28152g1
28336bl1
28566z1
28798a1
29015d1
29238i1
29435c1
29673f1
29901d1
30144y1
30350h1
30576n1
3081

215424i1
215634c1
215880k1
216137a1
216370a1
216549c1
216752o1
216980b1
217200dx1
217413bd1
217674a1
217854co1
218064q1
218320d1
218560a1
218785h1
218988l1
219198l1
219450ea1
219660f1
219921a1
220138a1
220372b1
220608bs1
220825c1
221056c1
221265b1
221502n1
221760ea1
221920b1
222144cb1
222400f1
222630g1
222816b1
223075b1
223278j1
223497f1
223686bp1
223930a1
224181c1
224400id1
224640i1
224880bb1
225144j1
225354r1
225600cb1
225822a1
226080g1
226330e1
226575f1
226800ca1
227010e1
227205d1
227443b1
227690b1
227910c1
228150ch1
228354f1
228600d1
228800dj1
229026bt1
229264f1
229488h1
229746bf1
229950ff1
230202i1
230434n1
230682a1
230913b1
231179c1
231360dt1
231616bq1
231856a1
232112d1
232350e1
232596k1
232830be1
233046cb1
233310bz1
233576g1
233800t1
234048k1
234240bc1
234465bq1
234696u1
234945g1
235200k1
235248dl1
235470bi1
235710j1
235950bv1
236130j1
236398c1
236640by1
236880ej1
237120ck1
237344a1
237600q1
237825l1
238050dc1
238266w1
238459c1
238702d1
238950bc1
239184r1
239424ca1
239725a1
2399

## The hash table is a dictionary: congruent curves will have the same hash (but possibly some incongruent curves will also have the same hash, if they have the same values of $a_p\pmod{13}$ for all $30$ primes tested).  So we extract sets of mutually congruent curves by looking at the dictionary's values (which are lists of labels) of length greater than one.

In [3]:
congruent_sets = [s for s in hashtab_13_30.values() if len(s)>1]
print("The number of nontrivial congruent sets is {}".format(len(congruent_sets)))
from collections import Counter
sizes = Counter()
for s in congruent_sets:
    sizes[len(s)] += 1
print("Distribution of sizes of nontrivial congruent sets: {}".format(sizes))

The number of nontrivial congruent sets is 151
Distribution of sizes of nontrivial congruent sets: Counter({2: 151})


### Test each of these pairs to see if we have an actual congruence, using Kraus's criterion (possibly after twisting to reduce the conductors).  With the default mumax=5000000 there is one pair for which this is not rigorous:  '49a1' and '78400gw1'.

In [4]:
bad_pairs = []
for s in congruent_sets:
    E1 = EllipticCurve(s[0])
    for r in s[1:]:
        E2 = EllipticCurve(r)
        res, info = test_cong(13,E1,E2, mumax=15000000, verbose=False)
        if not res:
            report(res,info,13,s[0],r)
            bad_pairs.append([s[0],r])
if bad_pairs:
    print("{} pairs of curves are not actually 13-congruent: {}".format(len(bad_pairs),bad_pairs))
else:
    print("all {} sets of curves really are 13-congruent".format(len(congruent_sets)))

Curves 49a1 and 78400gw1: testing ell up to 14837760 mod 13
Congruence mod 13 fails for 25921a1 and 78400gw1
 (not even isomorphic up to semisimplication: (29, 2, -2) )
1 pairs of curves are not actually 13-congruent: [['25921a1', '78400gw1']]


### So we have one "false positive" pair which we now remove from the list of congruent pairs:

In [5]:
bad_pair = bad_pairs[0]
assert bad_pair in congruent_sets
congruent_sets.remove(bad_pair)
print("After removing the false positive pair {}, there are {} congruent pairs remaining".format(bad_pair,len(congruent_sets)))

After removing the false positive pair ['25921a1', '78400gw1'], there are 150 congruent pairs remaining


### Now we reduce to a smaller set of pairs such that every congruent pair is a twist of one in the smaller set

In [6]:
j_pairs = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in congruent_sets]
j_sets = list(Set(j_pairs))
print("Out of {} pairs, there are only {} distinct pairs of j-invariants".format(len(j_pairs),len(j_sets)))

Out of 150 pairs, there are only 19 distinct pairs of j-invariants


In [7]:
congruent_pairs = []
for jset in j_sets:
    for s2 in congruent_sets:
        if jset==Set([EllipticCurve(s).j_invariant() for s in s2]):
            congruent_pairs.append(s2)
            break
print("{} congruent pairs up to twist: {}".format(len(congruent_pairs),congruent_pairs))

19 congruent pairs up to twist: [['11025u1', '143325dg1'], ['231231bl1', '231231bm1'], ['5175j1', '150075w1'], ['12274j1', '135014j1'], ['208c1', '3952c1'], ['83790dz1', '83790ea1'], ['17640y1', '194040dh1'], ['368186ca1', '368186cb1'], ['20184k1', '20184l1'], ['14175y1', '184275h1'], ['168948e1', '168948f1'], ['1190a1', '265370d1'], ['95370cl1', '95370cm1'], ['1445b1', '10115e1'], ['5070j1', '35490bg1'], ['11638t1', '151294f1'], ['27930cc1', '307230fb1'], ['98010bh1', '98010bi1'], ['314330i1', '314330j1']]


### Test whether all these are irreducible, i.e. have no $13$-isogenies:

In [8]:
for s2 in congruent_pairs:
    for s in s2:
        E = EllipticCurve(s)
        assert not E.isogenies_prime_degree(13)

#### In fact the only isogeny degrees among these curves are 2, 3:

In [10]:
sum([Set(EllipticCurve(s[0]).reducible_primes()) for s in congruent_pairs],Set())

{2, 3}

In [11]:
out = open("mod13pairs_uptotwist.m",'w')
out.write('pairs := [\\\n')
for s in congruent_pairs[:-1]:
    out.write('[EllipticCurve("{}"), EllipticCurve("{}")],\\\n'.format(s[0],s[1]))
s = congruent_pairs[-1]
out.write('[EllipticCurve("{}"), EllipticCurve("{}")]\\\n'.format(s[0],s[1]))
out.write('];\n')
out.close()